# Imports

In [31]:
import re
import pandas as pd
import numpy as np
import scipy
import requests
from bs4 import BeautifulSoup
from scipy.stats import linregress
from scipy import stats
import scipy.stats as stats
from scipy.stats import pearsonr, linregress
import statsmodels.api as sm
import pingouin as pg
from statsmodels.stats.multitest import multipletests
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
import warnings
import matplotlib.cbook
import matplotlib.dates as mdates
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import seaborn as sns
sns.set_theme(style="white")
# import calplot
# import july
import time
import html5lib

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import TransformedTargetRegressor, make_column_transformer, ColumnTransformer, make_column_selector
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_score, adjusted_rand_score,r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, roc_curve, precision_recall_curve, auc, confusion_matrix
from sklearn.model_selection import RepeatedKFold, cross_val_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree, DecisionTreeClassifier

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from xgboost import XGBRegressor, XGBRFClassifier

In [32]:
# Add project root to sys.path so you can import path_config regardless of notebook location
sys.path.append(str(Path.cwd().parents[2]))  # adjust depth as needed if notebook is deep in folders

from path_config import project_path, project_data_sources_path, project_data_exports_path

print(f'project_path: {project_path}')
print(f'project_data_sources_path: {project_data_sources_path}')
print(f'project_data_exports_path: {project_data_exports_path}')

project_path: /Users/chrisizenour/Library/CloudStorage/Dropbox/python/projects/football
project_data_sources_path: /Users/chrisizenour/Library/CloudStorage/Dropbox/python/projects/football/data/sources
project_data_exports_path: /Users/chrisizenour/Library/CloudStorage/Dropbox/python/projects/football/data/exports


In [33]:
# project_path = Path.cwd()
# project_path = Path(__file__).resolve().parent
# print(f'project_path: {project_path}')
# print('')
# project_data_sources_path = Path.cwd() / 'data' / 'sources'
# print(f'project_data_sources_path: {project_data_sources_path}')
# print('')
# project_data_exports_path = Path.cwd() / 'data' / 'exports'
# print(f'project_data_exports_path: {project_data_exports_path}')

# User Defined Functions

## Dataset Import and Wrangling

In [34]:
def load_game_results_dataset():
    df = pd.read_excel(
        project_data_sources_path / 'fbs_game_results.xlsx',
        sheet_name='Sheet1 (2)',
        # header=1,
        engine='openpyxl',
    )
    # df = df.iloc[:-2]
    print(f'Shape of dataframe: {df.shape[0]} rows, {df.shape[1]} columns')
    print(f'Seasons included: {df["season"].unique()}')
    print(f'Null game_dates: {df["game_date"].isnull().sum()}')
    return df

In [35]:
def load_passing_offense_dataset():
    df = pd.read_excel(
        project_data_sources_path / 'fbs_passing_offense_results.xlsx',
        sheet_name='Sheet1',
        # header=1,
        engine='openpyxl',
    )
    # df = df.iloc[:-2]
    print(f'Shape of dataframe: {df.shape[0]} rows, {df.shape[1]} columns')
    return df

In [36]:
def load_receiving_offense_dataset():
    df = pd.read_excel(
        project_data_sources_path / 'fbs_receiving_offense_results.xlsx',
        sheet_name='Sheet1',
        # header=1,
        engine='openpyxl',
    )
    # df = df.iloc[:-2]
    print(f'Shape of dataframe: {df.shape[0]} rows, {df.shape[1]} columns')
    return df

In [37]:
def load_rushing_offense_dataset():
    df = pd.read_excel(
        project_data_sources_path / 'fbs_rushing_offense_results.xlsx',
        sheet_name='Sheet1',
        # header=1,
        engine='openpyxl',
    )
    # df = df.iloc[:-2]
    print(f'Shape of dataframe: {df.shape[0]} rows, {df.shape[1]} columns')
    return df

In [38]:
def load_spotrac_original_download_dataset():
    df = pd.read_csv(
        project_data_sources_path / 'spotrac_salary_cap_data_df_original_download.csv',
        # sheet_name='Sheet1',
        # header=1,
        # engine='openpyxl',
    )
    # df = df.iloc[:-2]
    print(f'Shape of dataframe: {df.shape[0]} rows, {df.shape[1]} columns')
    return df

In [39]:
def load_nfl_season_records_original_download_dataset():
    df = pd.read_csv(
        project_data_sources_path / 'nfl_season_records_df_original_download.csv',
        # sheet_name='Sheet1',
        # header=1,
        # engine='openpyxl',
    )
    # df = df.iloc[:-2]
    print(f'Shape of dataframe: {df.shape[0]} rows, {df.shape[1]} columns')
    return df

In [40]:
spotrac_base_url = 'https://www.spotrac.com/nfl/{team}/overview/_/year/{season}/sort/cap_total'

spotrac_teams = ['buffalo-bills', 'miami-dolphins', 'new-england-patriots', 'new-york-jets',
                 'dallas-cowboys', 'new-york-giants', 'philadelphia-eagles', 'washington-commanders',
                 'baltimore-ravens', 'cincinnati-bengals', 'cleveland-browns', 'pittsburgh-steelers',
                 'chicago-bears', 'detroit-lions', 'green-bay-packers', 'minnesota-vikings',
                 'houston-texans', 'indianapolis-colts', 'jacksonville-jaguars', 'tennessee-titans',
                 'atlanta-falcons', 'carolina-panthers', 'new-orleans-saints', 'tampa-bay-buccaneers',
                 'denver-broncos',  'kansas-city-chiefs', 'las-vegas-raiders', 'los-angeles-chargers',
                 'arizona-cardinals', 'los-angeles-rams', 'san-francisco-49ers', 'seattle-seahawks']

spotrac_team_test = ['washington-commanders']

spotrac_seasons = [str(i) for i in range(2011, 2025)]

In [41]:
defense = ['CB', 'DE', 'DT', 'FS', 'ILB', 'LB', 'OLB', 'S', 'SS', 'WLB']
offense = ['C', 'FB', 'G', 'LT', 'OL', 'QB', 'RB', 'RT', 'T', 'TE', 'WR']
special_teams = ['LS', 'K', 'KR', 'P', 'PR']

In [42]:
position_groups = {
    "QB": 'quarterback',
    "RB": 'running_back',
    'FB': "running_back",
    'C': 'offensive_line',
    'G': 'offensive_line',
    'T': 'offensive_line',
    'LT': 'offensive_line',
    'RT': 'offensive_line',
    'OL': 'offensive_line',
    'TE': "tight_end",
    'WR': "wide_receiver",
    'CB': 'secondary',
    'FS': 'secondary',
    'SS': 'secondary',
    'S': 'secondary',
    'ILB': 'linebacker',
    'LB': 'linebacker',
    'OLB': 'linebacker',
    'WLB': 'linebacker',
    'DT': 'defensive_line',
    'DE': 'defensive_line',
    'LS': 'long_snapper',
    'K': 'kicker',
    'KR': 'kick_returner',
    'P': 'punter',
    'PR': 'punt_returner',
}

In [43]:
nfl_standings_base_url = 'https://www.nfl.com/standings/league/{season}/REG'

In [44]:
college_seasons = [int(i) for i in range(1990, 2024)]

In [45]:
# Filter games based on the season and date rules
def filter_games_by_date(game_results_df):
    # Convert date column to datetime if not already
    game_results_df['game_date'] = pd.to_datetime(game_results_df['game_date'])

    # Define a mask for seasons 1990-2001 to exclude games played on or after December 10th
    mask = (
        (game_results_df['season'] < 2002) &
        (game_results_df['game_date'] >= pd.to_datetime(game_results_df['season'].astype(str) + '-12-10'))
    )

    # Exclude games matching the mask for 1990-2001; include all others
    return game_results_df[~mask]

In [46]:
def clean_df_columns(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('__', '_')
    return df

In [47]:
def set_date_cols_as_datetime(df):
    df['game_date'] = pd.to_datetime(df['game_date'], errors='coerce')
    return df

In [48]:
def remove_team_rankings(df):
    df['winner'] = df['winner'].str.replace('(', '', regex=False)
    df['winner'] = df['winner'].str.replace(')', '', regex=False)
    df['winner'] = df['winner'].str.replace(r'^\d+\s+', '', regex=True)
    df['loser'] = df['loser'].str.replace('(', '', regex=False)
    df['loser'] = df['loser'].str.replace(')', '', regex=False)
    df['loser'] = df['loser'].str.replace(r'^\d+\s+', '', regex=True)
    return df

In [49]:
def set_categorical_cols(df):
    df['season'] = pd.Categorical(df['season'], ordered=True, categories=[int(i) for i in range(1990, 2025)])
    return df

In [50]:
def set_spotrac_categorical_cols(df):
    df['season'] = pd.Categorical(df['season'], ordered=True, categories=[int(i) for i in range(2011, 2025)])
    return df

In [51]:
def set_nfl_records_categorical_cols(df):
    df['season'] = pd.Categorical(df['season'], ordered=True, categories=[int(i) for i in range(2011, 2025)])
    return df

In [52]:
def clean_spotrac_dataset(df):
    # Clean the 'Player' column: Remove the first last_name and strip whitespace
    if 'Player' in df.columns:
        df['Player'] = df['Player'].astype(str).str.replace(r'^\w+\s+', '', regex=True).str.strip()

    # Clean columns with '%' symbols: Remove '%' and convert to float
    for col in df.columns:
        if 'pct' in col.lower():  # Check if column name contains 'pct'
            df[col] = (
                df[col]
                .astype(str)  # Ensure string format
                .str.replace(r'%', '', regex=True)  # Remove % symbol
                .replace('-', '0')  # Replace '-' with '0'
                .astype(float)  # Convert to float
            )

    # Clean currency and numeric columns: Remove '$', ',', and convert parentheses to negatives
    for col in ['Cap Hit', 'Dead Cap', 'Cash Total']:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)  # Ensure string format
                .str.replace(r'[\$,]', '', regex=True)  # Remove $ and commas
                .str.replace(r'\((.*?)\)', r'-\1', regex=True)  # Convert parentheses to negatives
                .replace('-', '0')  # Replace '-' with '0'
                .astype(float)  # Convert to float
                .fillna(0)
            )

    return df

In [53]:
def classify_spotrac_player_position(pos):
    if pos in defense:
        return 'defense'
    elif pos in offense:
        return 'offense'
    elif pos in special_teams:
        return 'special_team'
    else:
        return 'unknown'  # Handle any unexpected positions

In [54]:
# Define the function to classify player positions into groups
def classify_nfl_position_group(pos):
    position_groups = {
        "QB": 'quarterback',
        "RB": 'running_back',
        'FB': "running_back",
        'C': 'offensive_line',
        'G': 'offensive_line',
        'T': 'offensive_line',
        'LT': 'offensive_line',
        'RT': 'offensive_line',
        'OL': 'offensive_line',
        'TE': "tight_end",
        'WR': "wide_receiver",
        'CB': 'secondary',
        'FS': 'secondary',
        'SS': 'secondary',
        'S': 'secondary',
        'ILB': 'linebacker',
        'LB': 'linebacker',
        'OLB': 'linebacker',
        'WLB': 'linebacker',
        'DT': 'defensive_line',
        'DE': 'defensive_line',
        'LS': 'special_team',
        'K': 'special_team',
        'KR': 'special_team',
        'P': 'special_team',
        'PR': 'special_team',
    }
    return position_groups.get(pos, 'unknown')

# Import data

In [55]:
game_results_df = load_game_results_dataset()
game_results_df = clean_df_columns(game_results_df)
columns_to_fix = ['winner', 'loser']
for col in columns_to_fix:
    game_results_df[col] = game_results_df[col].str.replace('\xa0', ' ', regex=False)
game_results_df = set_date_cols_as_datetime(game_results_df)
game_results_df = remove_team_rankings(game_results_df)
game_results_df = set_categorical_cols(game_results_df)
game_results_df['season_game_number'] = game_results_df['season_game_number'].astype(int)
game_results_df['season_week'] = game_results_df['season_week'].astype(int)
game_results_df['season_game_num'] = game_results_df['season'].astype(str) + '_' + game_results_df['season_game_number'].astype(str)
game_results_df['winner_points'] = game_results_df['winner_points'].astype(float)
game_results_df['loser_points'] = game_results_df['loser_points'].astype(float)
game_results_df['game_date'] = pd.to_datetime(game_results_df['game_date'], errors='coerce')
game_results_df['tie'] = game_results_df['winner_points'] == game_results_df['loser_points']

game_results_df['index'] = game_results_df.index

game_results_df.to_csv(f'{project_data_exports_path}/game_results_df.csv')

game_results_df

Shape of dataframe: 26910 rows, 12 columns
Seasons included: [2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010
 2009 2008 2007 2006 2005 2004 2003 2002 2001 2000 1999 1998 1997 1996
 1995 1994 1993 1992 1991 1990 2024]
Null game_dates: 0


,season_game_number,season_week,game_date,game_time,game_day_of_week,winner,winner_points,winner_game_location_designator,loser,loser_points,notes,season,season_game_num,tie,index
0,1,1,2023-08-26,17:30:00,Sat,Jacksonville State,17.0,NaN,Texas-El Paso,14.0,NaN,2023,2023_1,False,0
1,2,1,2023-08-26,21:00:00,Sat,Louisiana Tech,22.0,NaN,Florida International,17.0,NaN,2023,2023_2,False,1
2,3,1,2023-08-26,19:00:00,Sat,Massachusetts,41.0,@,New Mexico State,30.0,NaN,2023,2023_3,False,2
3,4,1,2023-08-26,14:30:00,Sat,Notre Dame,42.0,N,Navy,3.0,NaN,2023,2023_4,False,3
4,5,1,2023-08-26,19:00:00,Sat,San Diego State,20.0,NaN,Ohio,13.0,NaN,2023,2023_5,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26905,915,21,2025-01-03,16:00:00,Fri,Texas State,30.0,N,North Texas,28.0,First Responder Bowl,2024,2024_915,False,26905
26906,916,21,2025-01-04,11:00:00,Sat,Buffalo,26.0,N,Liberty,7.0,Bahamas Bowl,2024,2024_916,False,26906
26907,917,22,2025-01-09,19:30:00,Thu,Notre Dame,27.0,N,Penn State,24.0,Orange Bowl,2024,2024_917,False,26907
26908,918,22,2025-01-10,19:30:00,Fri,Ohio State,28.0,N,Texas,14.0,Cotton Bowl,2024,2024_918,False,26908


In [56]:
# pd.read_html('https://en.wikipedia.org/wiki/List_of_undefeated_NCAA_Division_I_football_teams')[3]

In [57]:
# pd.read_html('https://en.wikipedia.org/wiki/List_of_undefeated_NCAA_Division_I_football_teams')[5]

In [58]:
passing_offense_df = load_passing_offense_dataset()
passing_offense_df = clean_df_columns(passing_offense_df)
passing_offense_df = set_categorical_cols(passing_offense_df)
passing_offense_df['season_game_num'] = passing_offense_df['season'].astype(str) + '_' + passing_offense_df['rk'].astype(str)
passing_offense_df = passing_offense_df.rename(columns={'cmp%': 'cmp_pct',
                                                        'td%': 'td_pct',
                                                        'int%': 'int_pct',
                                                        'y/a': 'yds_per_att',
                                                        'ay/a': 'avg_yds_per_att',
                                                        'y/c': 'yds_per_cmp',
                                                        'y/g': 'yds_per_g',})
passing_offense_df_float_cols = ['g', 'cmp', 'att', 'cmp_pct', 'yds', 'td', 'td_pct', 'int', 'int_pct', 'yds_per_att', 'avg_yds_per_att', 'yds_per_cmp', 'yds_per_g']
passing_offense_df[passing_offense_df_float_cols] = passing_offense_df[passing_offense_df_float_cols].astype(float)

passing_offense_df['index'] = passing_offense_df.index

passing_offense_df.to_csv(f'{project_data_exports_path}/passing_offense_df.csv')
passing_offense_df

Shape of dataframe: 16046 rows, 21 columns


,rk,player,team,conf,g,cmp,att,cmp_pct,yds,td,...,yds_per_att,avg_yds_per_att,yds_per_cmp,yds_per_g,rate,awards,player-additional,season,season_game_num,index
0,1,Michael Penix Jr.*,Washington,Pac-12,15.0,363.0,555.0,65.4,4903.0,36.0,...,8.8,9.24,13.5,326.9,157.1,H-2Maxwell,michael-penix-jr-1,2023,2023_1,0
1,2,Bo Nix*,Oregon,Pac-12,14.0,364.0,470.0,77.4,4508.0,45.0,...,9.6,11.22,12.4,322.0,188.3,H-3,bo-nix-1,2023,2023_2,1
2,3,Carson Beck*,Georgia,SEC,14.0,302.0,417.0,72.4,3941.0,24.0,...,9.5,9.95,13.0,281.5,167.9,NaN,carson-beck-1,2023,2023_3,2
3,4,Seth Henigan*,Memphis,American,13.0,318.0,476.0,66.8,3883.0,32.0,...,8.2,8.65,12.2,298.7,153.7,NaN,seth-henigan-1,2023,2023_4,3
4,5,Jayden Daniels*,LSU,SEC,12.0,236.0,327.0,72.2,3812.0,40.0,...,11.7,13.55,16.2,317.7,208.0,H-1AACamp,jayden-daniels-1,2023,2023_5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,506,Wilson Berry,Kentucky,SEC,12.0,0.0,1.0,0.0,0.0,0.0,...,0.0,-45.00,NaN,0.0,-200.0,NaN,wilson-berry-1,2024,2024_506,16041
16042,507,Xavier Restrepo*,Miami (FL),ACC,12.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.00,NaN,0.0,0.0,AA,xavier-restrepo-1,2024,2024_507,16042
16043,508,Zion Turner*,Jacksonville State,CUSA,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.00,NaN,0.0,0.0,NaN,zion-turner-1,2024,2024_508,16043
16044,509,Peter Moore*,Virginia Tech,ACC,13.0,1.0,1.0,100.0,-1.0,0.0,...,-1.0,-1.00,-1.0,-0.1,91.6,NaN,peter-moore-1,2024,2024_509,16044


In [59]:
receiving_offense_df = load_receiving_offense_dataset()
receiving_offense_df = clean_df_columns(receiving_offense_df)
receiving_offense_df = set_categorical_cols(receiving_offense_df)
receiving_offense_df['season_game_num'] = receiving_offense_df['season'].astype(str) + '_' + receiving_offense_df['rk'].astype(str)
receiving_offense_df = receiving_offense_df.rename(columns={'rec': 'receptions',
                                                            'yds': 'receiving_yds',
                                                            'y/r': 'yds_per_reception',
                                                            'td': 'receiving_td',
                                                            'y/g': 'receiving_yds_per_g',
                                                            'att': 'rushing_att',
                                                            'yds.1': 'rushing_yds',
                                                            'y/a': 'rushing_yds_per_rushing_att',
                                                            'td.1': 'rushing_td',
                                                            'y/g.1': 'rushing_yds_per_g',
                                                            'plays': 'scrimmage_plays',
                                                            'yds.2': 'scrimmage_yds',
                                                            'avg': 'avg_scrimmage_yds_per_play',
                                                            'td.2': 'scrimmage_td'
                                                            })
receiving_offense_df_float_cols = ['g', 'receptions', 'receiving_yds', 'yds_per_reception', 'receiving_td', 'receiving_yds_per_g', 'rushing_att', 'rushing_yds', 'rushing_yds_per_rushing_att', 'rushing_td', 'rushing_yds_per_g', 'scrimmage_plays', 'scrimmage_yds', 'avg_scrimmage_yds_per_play', 'scrimmage_td']
receiving_offense_df[receiving_offense_df_float_cols] = receiving_offense_df[receiving_offense_df_float_cols].astype(float)

receiving_offense_df['index'] = receiving_offense_df.index

receiving_offense_df.to_csv(f'{project_data_exports_path}/receiving_offense_df.csv')
receiving_offense_df

Shape of dataframe: 61793 rows, 22 columns


,rk,player,team,conf,g,receptions,receiving_yds,yds_per_reception,receiving_td,receiving_yds_per_g,...,rushing_yds_per_g,scrimmage_plays,scrimmage_yds,avg_scrimmage_yds_per_play,scrimmage_td,awards,player_identifier,season,season_game_num,index
0,1,Rome Odunze*,Washington,Pac-12,15.0,92.0,1640.0,17.8,13.0,109.3,...,2.5,94.0,1677.0,17.8,14.0,AA,rome-odunze-1,2023,2023_1,0
1,2,Malik Nabers*,LSU,SEC,13.0,89.0,1569.0,17.6,14.0,120.7,...,0.1,90.0,1570.0,17.4,14.0,AA,malik-nabers-1,2023,2023_2,1
2,3,Ricky White*,Nevada-Las Vegas,MWC,14.0,88.0,1483.0,16.9,8.0,105.9,...,0.1,89.0,1484.0,16.7,8.0,NaN,ricky-white-2,2023,2023_3,2
3,4,Malik Washington,Virginia,ACC,12.0,110.0,1426.0,13.0,9.0,118.8,...,0.6,113.0,1433.0,12.7,9.0,NaN,malik-washington-1,2023,2023_4,3
4,5,Tetairoa McMillan*,Arizona,Pac-12,13.0,90.0,1402.0,15.6,10.0,107.8,...,0.2,91.0,1405.0,15.4,10.0,NaN,tetairoa-mcmillan-1,2023,2023_5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61788,2155,John Long,Rice,American,8.0,1.0,-8.0,-8.0,0.0,-1.0,...,0.0,1.0,-8.0,-8.0,0.0,NaN,john-long-2,2024,2024_2155,61788
61789,2156,Riley Leonard*,Notre Dame,Ind,16.0,1.0,-8.0,-8.0,0.0,-0.5,...,56.6,185.0,898.0,4.9,17.0,NaN,riley-leonard-1,2024,2024_2156,61789
61790,2157,Jalen Kitna,UAB,American,8.0,1.0,-9.0,-9.0,0.0,-1.1,...,-7.5,46.0,-69.0,-1.5,0.0,NaN,jalen-kitna-1,2024,2024_2157,61790
61791,2158,Tristan Leigh,Clemson,ACC,12.0,1.0,-9.0,-9.0,0.0,-0.8,...,0.0,1.0,-9.0,-9.0,0.0,NaN,tristan-leigh-1,2024,2024_2158,61791


In [60]:
rushing_offense_df = load_rushing_offense_dataset()
rushing_offense_df = clean_df_columns(rushing_offense_df)
rushing_offense_df = set_categorical_cols(rushing_offense_df)
rushing_offense_df['season_game_num'] = rushing_offense_df['season'].astype(str) + '_' + rushing_offense_df['rk'].astype(str)
rushing_offense_df = rushing_offense_df.rename(columns={'att': 'rushing_att',
                                                        'yds': 'rushing_yds',
                                                        'y/a': 'rushing_yds_per_rushing_att',
                                                        'td': 'rushing_td',
                                                        'y/g': 'rushing_yds_per_g',
                                                        'rec': 'receptions',
                                                        'yds.1': 'receiving_yds',
                                                        'y/r': 'receiving_yds_per_reception',
                                                        'td.1': 'receiving_td',
                                                        'y/g.1': 'receiving_yds_per_g',
                                                        'plays': 'scrimmage_plays',
                                                            'yds.2': 'scrimmage_yds',
                                                            'avg': 'avg_scrimmage_yds_per_play',
                                                            'td.2': 'scrimmage_td'
                                                        })
rushing_offense_df_float_cols = ['g', 'rushing_att', 'rushing_yds', 'rushing_yds_per_rushing_att', 'rushing_td', 'rushing_yds_per_g', 'receptions', 'receiving_yds', 'receiving_yds_per_reception', 'receiving_td', 'receiving_yds_per_g', 'scrimmage_plays', 'scrimmage_yds', 'avg_scrimmage_yds_per_play', 'scrimmage_td']
rushing_offense_df[rushing_offense_df_float_cols] = rushing_offense_df[rushing_offense_df_float_cols].astype(float)

rushing_offense_df['index'] = rushing_offense_df.index

rushing_offense_df.to_csv(f'{project_data_exports_path}/rushing_offense_df.csv')
rushing_offense_df

Shape of dataframe: 50107 rows, 22 columns


,rk,player,team,conf,g,rushing_att,rushing_yds,rushing_yds_per_rushing_att,rushing_td,rushing_yds_per_g,...,receiving_yds_per_g,scrimmage_plays,scrimmage_yds,avg_scrimmage_yds_per_play,scrimmage_td,awards,player_identifier,season,season_game_num,index
0,1,Darren Lewis,Texas A&M,SWC,12.0,291.0,1691.0,5.8,18.0,140.9,...,4.0,297.0,1739.0,5.9,19.0,H-8AA,darren-lewis-1,1990,1990_1,0
1,2,Gerald Hudson,Oklahoma State,Big 8,11.0,279.0,1642.0,5.9,10.0,149.3,...,2.3,288.0,1667.0,5.8,10.0,NaN,gerald-hudson-1,1990,1990_2,1
2,3,Eric Bieniemy,Colorado,Big 8,11.0,288.0,1628.0,5.7,17.0,148.0,...,14.5,301.0,1787.0,5.9,17.0,H-3AA,eric-bieniemy-1,1990,1990_3,2
3,4,Tico Duckett,Michigan State,Big Ten,11.0,249.0,1376.0,5.5,10.0,125.1,...,5.6,258.0,1438.0,5.6,11.0,NaN,tico-duckett-1,1990,1990_4,3
4,5,Roger Grant,Utah State,Big West,11.0,266.0,1370.0,5.2,8.0,124.5,...,2.3,271.0,1395.0,5.1,8.0,NaN,roger-grant-1,1990,1990_5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50102,1653,Connor Bazelak*,Bowling Green,MAC,13.0,48.0,-91.0,-1.9,2.0,-7.0,...,0.0,48.0,-91.0,-1.9,2.0,NaN,connor-bazelak-1,2024,2024_1653,50102
50103,1654,Hayden Wolff*,Western Michigan,MAC,13.0,39.0,-101.0,-2.6,0.0,-7.8,...,0.0,39.0,-101.0,-2.6,0.0,NaN,hayden-wolff-1,2024,2024_1654,50103
50104,1655,Deshawn Purdie,Charlotte,American,9.0,41.0,-130.0,-3.2,1.0,-14.4,...,0.0,41.0,-130.0,-3.2,1.0,NaN,deshawn-purdie-1,2024,2024_1655,50104
50105,1656,Evan Bullock*,Louisiana Tech,CUSA,12.0,63.0,-130.0,-2.1,1.0,-10.8,...,0.0,63.0,-130.0,-2.1,1.0,NaN,evan-bullock-1,2024,2024_1656,50105


In [61]:
# DataFrame to store all data
# spotrac_salary_cap_data_df_original_download = pd.DataFrame()
#
# # Loop through teams and seasons
# for team in spotrac_teams:
#     for season in spotrac_seasons:
#         try:
#             # Construct URL
#             url = spotrac_base_url.format(team=team, season=season)
#             print(f"Fetching data for {team} in {season} from {url}")
#
#             # Read HTML tables
#             tables = pd.read_html(url)
#
#             # Exclude the last table
#             tables_to_scrape = tables[:-1]  # All tables except the last one
#
#             # Loop through the selected tables
#             for idx, table in enumerate(tables_to_scrape):
#                 # Clean column names: Remove (xx) from Player column
#                 table.columns = [re.sub(r'\s*\(\d+\)', '', col) for col in table.columns]
#
#                 # Rename 'Unnamed: 0' to 'table_row_number' if it exists
#                 if 'Unnamed: 0' in table.columns:
#                     table.rename(columns={'Unnamed: 0': 'table_row_number'}, inplace=True)
#
#                 # Add team, season, and table metadata columns
#                 table['team'] = team
#                 table['season'] = season
#                 table['table_number'] = idx
#
#                 # Append to the main DataFrame
#                 spotrac_salary_cap_data_df_original_download = pd.concat([spotrac_salary_cap_data_df_original_download, table], ignore_index=True)
#
#             # Add a delay between requests
#             time.sleep(3)  # Adjust the delay as necessary (e.g., 3 seconds)
#
#         except Exception as e:
#             print(f"Error fetching data for {team} in {season}: {e}")
#
# # Save to a CSV file or process further
# spotrac_salary_cap_data_df_original_download.to_csv(f"{project_data_sources_path}/spotrac_salary_cap_data_df_original_download.csv", index=False)
# print("Data scraping complete. Saved to spotrac_salary_cap_data_df_original_download.csv.")
#
# spotrac_salary_cap_data_df_original_download

In [62]:
spotrac_salary_cap_data_df = load_spotrac_original_download_dataset()
spotrac_salary_cap_data_df = clean_spotrac_dataset(spotrac_salary_cap_data_df)
spotrac_salary_cap_data_df = clean_df_columns(spotrac_salary_cap_data_df)
spotrac_salary_cap_data_df = set_spotrac_categorical_cols(spotrac_salary_cap_data_df)
spotrac_salary_cap_data_df['position_level_one'] = spotrac_salary_cap_data_df['pos'].apply(classify_spotrac_player_position)

# Apply the function to classify positions
if 'pos' in spotrac_salary_cap_data_df.columns:
    spotrac_salary_cap_data_df['position_level_two'] = (
        spotrac_salary_cap_data_df['pos'].apply(classify_nfl_position_group)
    )

spotrac_salary_cap_data_df = spotrac_salary_cap_data_df.loc[spotrac_salary_cap_data_df['position_level_two'] != "unknown", :]
spotrac_salary_cap_data_df = spotrac_salary_cap_data_df.loc[spotrac_salary_cap_data_df['cap_hit_pct_league_cap'].notnull(), :]
spotrac_salary_cap_data_df = spotrac_salary_cap_data_df.drop(columns=['cap_hit_top-51', 'round_/_pick'])
spotrac_salary_cap_data_df['roster_status'] = spotrac_salary_cap_data_df['table_number'].apply(lambda x: 'active' if x == 0 else 'inactive')

spotrac_salary_cap_data_df = spotrac_salary_cap_data_df.reset_index(drop = True)

spotrac_salary_cap_data_df['spotrac_index'] = spotrac_salary_cap_data_df.index

spotrac_salary_cap_data_df.to_csv(f"{project_data_exports_path}/spotrac_salary_cap_data_df.csv")
spotrac_salary_cap_data_df

Shape of dataframe: 59836 rows, 14 columns


,table_row_number,player,pos,age,cap_hit,cap_hit_pct_league_cap,dead_cap,cash_total,free_agent_year,team,season,table_number,position_level_one,position_level_two,roster_status,spotrac_index
0,1.0,Ryan Fitzpatrick,QB,29.0,5610000.0,4.66,0.0,13220000.0,2012.0,buffalo-bills,2011,0,offense,quarterback,active,0
1,2.0,Chris Kelsay,DE,32.0,4500000.0,3.74,0.0,2100000.0,2015.0,buffalo-bills,2011,0,defense,defensive_line,active,1
2,3.0,Drayton Florence,CB,31.0,4166667.0,3.46,0.0,5500000.0,2014.0,buffalo-bills,2011,0,defense,secondary,active,2
3,4.0,Spencer Johnson,DT,30.0,4000000.0,3.32,0.0,3500000.0,2013.0,buffalo-bills,2011,0,defense,defensive_line,active,3
4,5.0,Dwan Edwards,DT,30.0,3900000.0,3.24,0.0,3900000.0,2013.0,buffalo-bills,2011,0,defense,defensive_line,active,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59352,97.0,Lance Boykin,CB,24.0,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,2024,3,defense,secondary,inactive,59352
59353,98.0,James Campbell,CB,25.0,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,2024,3,defense,secondary,inactive,59353
59354,99.0,C.J. Johnson,WR,NaN,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,2024,3,offense,wide_receiver,inactive,59354
59355,100.0,Tyjon Lindsey,WR,25.0,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,2024,3,offense,wide_receiver,inactive,59355


In [63]:
# nfl_season_records_df_original_download = pd.DataFrame()
#
# # Columns to split
# columns_to_split = ['Home', 'Road', 'Div', 'Conf', 'Non-Conf', 'Last 5']
#
# # Loop through each season and scrape data
# for season in spotrac_seasons:
#     try:
#         print(f"Processing season: {season}")
#         # Construct URL for the season
#         url = nfl_standings_base_url.format(season=season)
#         print(f"Fetching data for {season} from {url}")
#
#         # Read tables from the URL
#         tables = pd.read_html(url)
#
#         # Combine all tables into a single DataFrame and add the season column
#         for table in tables:
#             table['season'] = season  # Add the season as a new column
#             nfl_season_records_df_original_download = pd.concat(
#                 [nfl_season_records_df_original_download, table],
#                 ignore_index=True
#             )
#
#     except Exception as e:
#         print(f"Error fetching data for {season}: {e}")
#
# # Rename columns
# rename_columns = {
#     'Pct': 'Div_win_pct',
#     'Pct.1': 'Conf_win_pct'
# }
#
# nfl_season_records_df_original_download.rename(columns=rename_columns, inplace=True)
#
# # Clean the 'NFL Team' column to keep only the official team name
# if 'NFL Team' in nfl_season_records_df_original_download.columns:
#     nfl_season_records_df_original_download['NFL Team'] = (
#         nfl_season_records_df_original_download['NFL Team']
#         .str.split('  ', n=1, expand=True)[0]  # Split by two spaces and keep the left part
#     )
#
# # Split specified columns into wins, losses, and ties
# for col in columns_to_split:
#     if col in nfl_season_records_df_original_download.columns:
#         nfl_season_records_df_original_download[[f"{col}_wins", f"{col}_losses", f"{col}_ties"]] = (
#             nfl_season_records_df_original_download[col]
#             .astype(str)  # Ensure string format for splitting
#             .str.split(' - ', expand=True)  # Split into three parts
#         )
#
#         # Convert the new columns to integers
#         nfl_season_records_df_original_download[f"{col}_wins"] = nfl_season_records_df_original_download[f"{col}_wins"].astype(int)
#         nfl_season_records_df_original_download[f"{col}_losses"] = nfl_season_records_df_original_download[f"{col}_losses"].astype(int)
#         nfl_season_records_df_original_download[f"{col}_ties"] = nfl_season_records_df_original_download[f"{col}_ties"].astype(int)
#
# nfl_season_records_df_original_download.to_csv(f'{project_data_sources_path}/nfl_season_records_df_original_download.csv', index=False)
# print(f"Data scraping complete. Saved to nfl_season_records_df_original_download.csv")
#
# nfl_season_records_df_original_download

In [64]:
nfl_season_records_df = load_nfl_season_records_original_download_dataset()
nfl_season_records_df = clean_df_columns(nfl_season_records_df)
nfl_season_records_df['nfl_team_name'] = nfl_season_records_df['nfl_team'].str.lower().str.replace(' ', '-', regex=False)
nfl_season_records_df = set_nfl_records_categorical_cols(nfl_season_records_df)

nfl_season_records_df['nfl_season_records_index'] = nfl_season_records_df.index

nfl_season_records_df.to_csv(f'{project_data_exports_path}/nfl_season_records_df.csv')
nfl_season_records_df

Shape of dataframe: 448 rows, 36 columns


,nfl_team,w,l,t,pct,pf,pa,net_pts,home,road,...,conf_losses,conf_ties,non-conf_wins,non-conf_losses,non-conf_ties,last_5_wins,last_5_losses,last_5_ties,nfl_team_name,nfl_season_records_index
0,Indianapolis Colts,2,14,0,0.125,243,430,-187,2 - 6 - 0,0 - 8 - 0,...,10,0,0,4,0,2,3,0,indianapolis-colts,0
1,Los Angeles Rams,2,14,0,0.125,193,407,-214,1 - 7 - 0,1 - 7 - 0,...,11,0,1,3,0,0,5,0,los-angeles-rams,1
2,Minnesota Vikings,3,13,0,0.188,340,449,-109,1 - 7 - 0,2 - 6 - 0,...,9,0,0,4,0,1,4,0,minnesota-vikings,2
3,Cleveland Browns,4,12,0,0.250,218,307,-89,3 - 5 - 0,1 - 7 - 0,...,9,0,1,3,0,0,5,0,cleveland-browns,3
4,Tampa Bay Buccaneers,4,12,0,0.250,287,494,-207,3 - 5 - 0,1 - 7 - 0,...,9,0,1,3,0,0,5,0,tampa-bay-buccaneers,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,Buffalo Bills,13,4,0,0.765,525,368,157,8 - 0 - 0,5 - 4 - 0,...,3,0,4,1,0,3,2,0,buffalo-bills,443
444,Minnesota Vikings,14,3,0,0.824,432,332,100,8 - 1 - 0,6 - 2 - 0,...,3,0,5,0,0,4,1,0,minnesota-vikings,444
445,Philadelphia Eagles,14,3,0,0.824,463,303,160,8 - 1 - 0,6 - 2 - 0,...,3,0,5,0,0,4,1,0,philadelphia-eagles,445
446,Detroit Lions,15,2,0,0.882,564,342,222,7 - 2 - 0,8 - 0 - 0,...,1,0,4,1,0,4,1,0,detroit-lions,446


# Merge NFL Records with Spotrac Data

In [65]:
spotrac_nfl_records_df = (pd.merge(
    spotrac_salary_cap_data_df, nfl_season_records_df,
    how='left',
    left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
                          # .drop(columns=['index_x', 'index_y'])
                          )
spotrac_nfl_records_df['index'] = spotrac_nfl_records_df.index
spotrac_nfl_records_df.to_csv(f"{project_data_exports_path}/spotrac_nfl_records_df.csv")
spotrac_nfl_records_df

,table_row_number,player,pos,age,cap_hit,cap_hit_pct_league_cap,dead_cap,cash_total,free_agent_year,team,...,conf_ties,non-conf_wins,non-conf_losses,non-conf_ties,last_5_wins,last_5_losses,last_5_ties,nfl_team_name,nfl_season_records_index,index
0,1.0,Ryan Fitzpatrick,QB,29.0,5610000.0,4.66,0.0,13220000.0,2012.0,buffalo-bills,...,0,2,2,0,1,4,0,buffalo-bills,7,0
1,2.0,Chris Kelsay,DE,32.0,4500000.0,3.74,0.0,2100000.0,2015.0,buffalo-bills,...,0,2,2,0,1,4,0,buffalo-bills,7,1
2,3.0,Drayton Florence,CB,31.0,4166667.0,3.46,0.0,5500000.0,2014.0,buffalo-bills,...,0,2,2,0,1,4,0,buffalo-bills,7,2
3,4.0,Spencer Johnson,DT,30.0,4000000.0,3.32,0.0,3500000.0,2013.0,buffalo-bills,...,0,2,2,0,1,4,0,buffalo-bills,7,3
4,5.0,Dwan Edwards,DT,30.0,3900000.0,3.24,0.0,3900000.0,2013.0,buffalo-bills,...,0,2,2,0,1,4,0,buffalo-bills,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59352,97.0,Lance Boykin,CB,24.0,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,...,0,4,1,0,3,2,0,seattle-seahawks,437,59352
59353,98.0,James Campbell,CB,25.0,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,...,0,4,1,0,3,2,0,seattle-seahawks,437,59353
59354,99.0,C.J. Johnson,WR,NaN,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,...,0,4,1,0,3,2,0,seattle-seahawks,437,59354
59355,100.0,Tyjon Lindsey,WR,25.0,667.0,0.00,0.0,0.0,NaN,seattle-seahawks,...,0,4,1,0,3,2,0,seattle-seahawks,437,59355


## Subsidiary NFL Record and Spotrac Dataframes

In [66]:
spotrac_nfl_team_season_roster_df = spotrac_nfl_records_df.groupby([
    'team', 'season', 'roster_status'
], observed=True).agg({
    'index': 'count',
    'cap_hit': 'sum'
}).reset_index().rename(columns={'index': 'player_count', 'cap_hit': 'cap_hit_sum'})

# compute proportions within each team‐season
spotrac_nfl_team_season_roster_df['player_count_prop'] = (
    spotrac_nfl_team_season_roster_df['player_count'] / spotrac_nfl_team_season_roster_df.groupby(['team', 'season'], observed=True)['player_count'].transform('sum')
)

spotrac_nfl_team_season_roster_df['cap_hit_prop'] = (
    spotrac_nfl_team_season_roster_df['cap_hit_sum'] / spotrac_nfl_team_season_roster_df.groupby(['team', 'season'], observed=True)['cap_hit_sum'].transform('sum')
)

spotrac_nfl_team_season_roster_df = pd.merge(spotrac_nfl_team_season_roster_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_season_roster_df = spotrac_nfl_team_season_roster_df.drop(['nfl_team_name'], axis=1)

In [67]:
spotrac_nfl_team_season_roster_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_season_roster_df.csv')
spotrac_nfl_team_season_roster_df

,team,season,roster_status,player_count,cap_hit_sum,player_count_prop,cap_hit_prop,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,active,58,111028866.0,0.716049,0.937187,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,inactive,23,7441498.0,0.283951,0.062813,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2012,active,53,88769403.0,0.638554,0.753574,5,11,0.313,250,357,-107,0.167,0.250
3,arizona-cardinals,2012,inactive,30,29028393.0,0.361446,0.246426,5,11,0.313,250,357,-107,0.167,0.250
4,arizona-cardinals,2013,active,52,92369539.0,0.495238,0.769450,10,6,0.625,379,324,55,0.333,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,washington-commanders,2022,inactive,86,47974051.0,0.618705,0.230569,8,8,0.500,321,343,-22,0.417,0.458
892,washington-commanders,2023,active,53,133719895.0,0.339744,0.610152,4,13,0.235,329,518,-189,0.000,0.167
893,washington-commanders,2023,inactive,103,85438357.0,0.660256,0.389848,4,13,0.235,329,518,-189,0.000,0.167
894,washington-commanders,2024,active,53,186228005.0,0.341935,0.773911,12,5,0.706,485,391,94,0.667,0.750


In [68]:
spotrac_nfl_team_season_roster_pos_df = spotrac_nfl_records_df.groupby([
    'team', 'season', 'pos', 'roster_status'
], observed=True).agg({
    'index': 'count',
    'cap_hit': 'sum'
}).reset_index().rename(columns={'index': 'player_count', 'cap_hit': 'cap_hit_sum'})

# compute each pos’s share within the team–season–roster_status group
spotrac_nfl_team_season_roster_pos_df_denoms_count = test_2_df.groupby(
    ['team','season','roster_status'], observed=True
)['player_count'].transform('sum')

spotrac_nfl_team_season_roster_pos_df_denoms_cap = test_2_df.groupby(
    ['team','season','roster_status'], observed=True
)['cap_hit_sum'].transform('sum')

spotrac_nfl_team_season_roster_pos_df['pos_count_prop'] = spotrac_nfl_team_season_roster_pos_df['player_count'] / spotrac_nfl_team_season_roster_pos_df_denoms_count
spotrac_nfl_team_season_roster_pos_df['pos_cap_hit_prop'] = spotrac_nfl_team_season_roster_pos_df['cap_hit_sum'] / spotrac_nfl_team_season_roster_pos_df_denoms_cap

spotrac_nfl_team_season_roster_pos_df = pd.merge(spotrac_nfl_team_season_roster_pos_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_season_roster_pos_df = spotrac_nfl_team_season_roster_pos_df.drop(['nfl_team_name'], axis=1)

NameError: name 'test_2_df' is not defined

In [102]:
spotrac_nfl_team_season_roster_pos_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_season_roster_pos_df.csv')
spotrac_nfl_team_season_roster_pos_df

,team,season,pos,roster_status,player_count,cap_hit_sum,pos_count_prop,pos_cap_hit_prop,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,C,active,1,2350000.0,0.017241,0.021166,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,CB,active,6,8685818.0,0.103448,0.078230,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,CB,inactive,5,1320743.0,0.217391,0.177483,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2011,DE,active,4,7895000.0,0.068966,0.071108,8,8,0.500,312,348,-36,0.667,0.583
4,arizona-cardinals,2011,DE,inactive,1,478000.0,0.043478,0.064234,8,8,0.500,312,348,-36,0.667,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16453,washington-commanders,2024,T,inactive,3,248334.0,0.029412,0.004565,12,5,0.706,485,391,94,0.667,0.750
16454,washington-commanders,2024,TE,active,3,7022541.0,0.056604,0.037709,12,5,0.706,485,391,94,0.667,0.750
16455,washington-commanders,2024,TE,inactive,7,2909721.0,0.068627,0.053483,12,5,0.706,485,391,94,0.667,0.750
16456,washington-commanders,2024,WR,active,6,29557825.0,0.113208,0.158718,12,5,0.706,485,391,94,0.667,0.750


In [103]:
spotrac_nfl_team_season_roster_position_level_one_df = spotrac_nfl_records_df.groupby([
    'team', 'season', 'position_level_one', 'roster_status'
], observed=True).agg({
    'index': 'count',
    'cap_hit': 'sum'
}).reset_index().rename(columns={'index': 'player_count', 'cap_hit': 'cap_hit_sum'})

# compute each pos’s share within the team–season–roster_status group
spotrac_nfl_team_season_roster_position_level_one_df_denoms_count = test_3_df.groupby(
    ['team','season','roster_status'], observed=True
)['player_count'].transform('sum')

spotrac_nfl_team_season_roster_position_level_one_df_denoms_cap = test_3_df.groupby(
    ['team','season','roster_status'], observed=True
)['cap_hit_sum'].transform('sum')

spotrac_nfl_team_season_roster_position_level_one_df['position_level_one_count_prop'] = spotrac_nfl_team_season_roster_position_level_one_df['player_count'] / spotrac_nfl_team_season_roster_position_level_one_df_denoms_count
spotrac_nfl_team_season_roster_position_level_one_df['position_level_one_cap_hit_prop'] = spotrac_nfl_team_season_roster_position_level_one_df['cap_hit_sum'] / spotrac_nfl_team_season_roster_position_level_one_df_denoms_cap

spotrac_nfl_team_season_roster_position_level_one_df = pd.merge(spotrac_nfl_team_season_roster_position_level_one_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_season_roster_position_level_one_df = spotrac_nfl_team_season_roster_position_level_one_df.drop(['nfl_team_name'], axis=1)

In [104]:
spotrac_nfl_team_season_roster_position_level_one_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_season_roster_position_level_one_df.csv')
spotrac_nfl_team_season_roster_position_level_one_df

,team,season,position_level_one,roster_status,player_count,cap_hit_sum,position_level_one_count_prop,position_level_one_cap_hit_prop,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,defense,active,26,46212697.0,0.448276,0.416222,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,defense,inactive,12,3327689.0,0.521739,0.447180,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,offense,active,29,61041169.0,0.500000,0.549777,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2011,offense,inactive,10,3969692.0,0.434783,0.533453,8,8,0.500,312,348,-36,0.667,0.583
4,arizona-cardinals,2011,special_team,active,3,3775000.0,0.051724,0.034000,8,8,0.500,312,348,-36,0.667,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,washington-commanders,2024,defense,inactive,39,15942070.0,0.382353,0.293029,12,5,0.706,485,391,94,0.667,0.750
2601,washington-commanders,2024,offense,active,26,87344973.0,0.490566,0.469022,12,5,0.706,485,391,94,0.667,0.750
2602,washington-commanders,2024,offense,inactive,56,35648630.0,0.549020,0.655252,12,5,0.706,485,391,94,0.667,0.750
2603,washington-commanders,2024,special_team,active,3,5441944.0,0.056604,0.029222,12,5,0.706,485,391,94,0.667,0.750


In [105]:
spotrac_nfl_team_season_roster_position_level_two_df = spotrac_nfl_records_df.groupby([
    'team', 'season', 'position_level_two', 'roster_status'
], observed=True).agg({
    'index': 'count',
    'cap_hit': 'sum'
}).reset_index().rename(columns={'index': 'player_count', 'cap_hit': 'cap_hit_sum'})

# compute each pos’s share within the team–season–roster_status group
spotrac_nfl_team_season_roster_position_level_two_df_denoms_count = test_4_df.groupby(
    ['team','season','roster_status'], observed=True
)['player_count'].transform('sum')

spotrac_nfl_team_season_roster_position_level_two_df_denoms_cap = test_4_df.groupby(
    ['team','season','roster_status'], observed=True
)['cap_hit_sum'].transform('sum')

spotrac_nfl_team_season_roster_position_level_two_df['position_level_two_count_prop'] = spotrac_nfl_team_season_roster_position_level_two_df['player_count'] / spotrac_nfl_team_season_roster_position_level_two_df_denoms_count
spotrac_nfl_team_season_roster_position_level_two_df['position_level_two_cap_hit_prop'] = spotrac_nfl_team_season_roster_position_level_two_df['cap_hit_sum'] / spotrac_nfl_team_season_roster_position_level_two_df_denoms_cap

spotrac_nfl_team_season_roster_position_level_two_df = pd.merge(spotrac_nfl_team_season_roster_position_level_two_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_season_roster_position_level_two_df = spotrac_nfl_team_season_roster_position_level_two_df.drop(['nfl_team_name'], axis=1)

In [106]:
spotrac_nfl_team_season_roster_position_level_two_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_season_roster_position_level_two_df.csv')
spotrac_nfl_team_season_roster_position_level_two_df

,team,season,position_level_two,roster_status,player_count,cap_hit_sum,position_level_two_count_prop,position_level_two_cap_hit_prop,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,defensive_line,active,7,11140966.0,0.120690,0.100343,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,defensive_line,inactive,3,567400.0,0.130435,0.076248,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,linebacker,active,8,12766149.0,0.137931,0.114980,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2011,linebacker,inactive,4,1439546.0,0.173913,0.193448,8,8,0.500,312,348,-36,0.667,0.583
4,arizona-cardinals,2011,offensive_line,active,8,21546666.0,0.137931,0.194064,8,8,0.500,312,348,-36,0.667,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7888,washington-commanders,2024,special_team,inactive,7,2813751.0,0.068627,0.051719,12,5,0.706,485,391,94,0.667,0.750
7889,washington-commanders,2024,tight_end,active,3,7022541.0,0.056604,0.037709,12,5,0.706,485,391,94,0.667,0.750
7890,washington-commanders,2024,tight_end,inactive,7,2909721.0,0.068627,0.053483,12,5,0.706,485,391,94,0.667,0.750
7891,washington-commanders,2024,wide_receiver,active,6,29557825.0,0.113208,0.158718,12,5,0.706,485,391,94,0.667,0.750


In [252]:
spotrac_nfl_team_pos_level_two_grouped_df = spotrac_nfl_records_df.groupby(['team', 'season', 'position_level_two'], observed=True).agg({
    'index': 'count',
    'cap_hit_pct_league_cap': 'sum'
}).reset_index()
spotrac_nfl_team_pos_level_two_grouped_df = spotrac_nfl_team_pos_level_two_grouped_df.rename(columns={'index': 'player_count',
                                                                                                      'cap_hit_pct_league_cap': 'cap_hit_pct_sum'})

spotrac_nfl_team_pos_level_two_grouped_df = pd.merge(spotrac_nfl_team_pos_level_two_grouped_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_pos_level_two_grouped_df = spotrac_nfl_team_pos_level_two_grouped_df.drop(['nfl_team_name'], axis=1)

spotrac_nfl_team_pos_level_two_grouped_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_pos_level_two_grouped_df.csv')
spotrac_nfl_team_pos_level_two_grouped_df

,team,season,position_level_two,player_count,cap_hit_pct_sum,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,defensive_line,10,9.73,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,linebacker,12,11.81,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,offensive_line,12,19.32,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2011,quarterback,6,9.73,8,8,0.500,312,348,-36,0.667,0.583
4,arizona-cardinals,2011,running_back,9,4.22,8,8,0.500,312,348,-36,0.667,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,washington-commanders,2024,running_back,13,3.05,12,5,0.706,485,391,94,0.667,0.750
4028,washington-commanders,2024,secondary,30,11.62,12,5,0.706,485,391,94,0.667,0.750
4029,washington-commanders,2024,special_team,10,3.23,12,5,0.706,485,391,94,0.667,0.750
4030,washington-commanders,2024,tight_end,10,3.87,12,5,0.706,485,391,94,0.667,0.750


In [253]:
spotrac_nfl_team_pos_level_one_grouped_df = spotrac_nfl_records_df.groupby(['team', 'season', 'position_level_one'], observed=True).agg({'index': 'count',
                                                                                             'cap_hit_pct_league_cap': 'sum'}).reset_index()
spotrac_nfl_team_pos_level_one_grouped_df = spotrac_nfl_team_pos_level_one_grouped_df.rename(columns={'index': 'player_count',
                                                                                                      'cap_hit_pct_league_cap': 'cap_hit_pct_sum'})

spotrac_nfl_team_pos_level_one_grouped_df = pd.merge(spotrac_nfl_team_pos_level_one_grouped_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_pos_level_one_grouped_df = spotrac_nfl_team_pos_level_one_grouped_df.drop(['nfl_team_name'], axis=1)

spotrac_nfl_team_pos_level_one_grouped_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_pos_level_one_grouped_df.csv')
spotrac_nfl_team_pos_level_one_grouped_df

,team,season,position_level_one,player_count,cap_hit_pct_sum,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,defense,38,41.17,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,offense,39,54.03,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,special_team,4,3.26,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2012,defense,31,42.75,5,11,0.313,250,357,-107,0.167,0.250
4,arizona-cardinals,2012,offense,48,52.18,5,11,0.313,250,357,-107,0.167,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,washington-commanders,2023,offense,72,42.98,4,13,0.235,329,518,-189,0.000,0.167
1340,washington-commanders,2023,special_team,6,3.04,4,13,0.235,329,518,-189,0.000,0.167
1341,washington-commanders,2024,defense,63,42.81,12,5,0.706,485,391,94,0.667,0.750
1342,washington-commanders,2024,offense,82,48.15,12,5,0.706,485,391,94,0.667,0.750


In [254]:
# ---------------------------
# B) Compute Team–Season Totals
# ---------------------------
# 1. Total cap_hit for each team–season (active + inactive)
spotrac_nfl_team_totals = (
    spotrac_nfl_records_df.groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
      .sum()
      .rename(columns={'cap_hit': 'cap_hit_total'})
)

# 2. Active cap_hit total (table_number == 0)
spotrac_nfl_team_active = (
    spotrac_nfl_records_df[spotrac_nfl_records_df['table_number'] == 0]
    .groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_active'})
)

# 3. Inactive cap_hit total (table_number > 0)
spotrac_nfl_team_inactive = (
    spotrac_nfl_records_df[spotrac_nfl_records_df['table_number'] > 0]
    .groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_inactive'})
)

# Merge team totals into a single DataFrame.
# Instead of a blanket fillna(0) which affects categorical columns, fill numeric columns only.
df_team = (
    spotrac_nfl_team_totals
    .merge(spotrac_nfl_team_active, on=['team', 'season'], how='left')
    .merge(spotrac_nfl_team_inactive, on=['team', 'season'], how='left')
).copy()

# Fill missing numeric columns with 0 only.
num_cols = df_team.select_dtypes(include=['number']).columns
df_team[num_cols] = df_team[num_cols].fillna(0)


# ---------------------------
# C) Compute Position-Level Sums by Roster Status
# ---------------------------
# Group by team, season, position, position_level_one, position_level_two, and roster_status,
# then sum the cap_hit at the position level.
spotrac_nfl_records_pos = (
    spotrac_nfl_records_df.groupby(
        ['team', 'season', 'pos', 'position_level_one', 'position_level_two', 'roster_status'],
        as_index=False, observed=True
    )['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'pos_cap_hit_status'})
)

# Pivot so that we have separate columns for active and inactive sums per position
spotrac_nfl_records_pos_pivot = spotrac_nfl_records_pos.pivot_table(
    index=['team', 'season', 'pos', 'position_level_one', 'position_level_two'],
    columns='roster_status',
    values='pos_cap_hit_status',
    fill_value=0
).reset_index()
spotrac_nfl_records_pos_pivot.columns.name = None

# Rename the pivot columns for clarity
spotrac_nfl_records_pos_pivot = spotrac_nfl_records_pos_pivot.rename(columns={'active': 'pos_cap_hit_active', 'inactive': 'pos_cap_hit_inactive'})

# Create a new column for the total position cap_hit (active + inactive)
spotrac_nfl_records_pos_pivot['pos_cap_hit'] = spotrac_nfl_records_pos_pivot['pos_cap_hit_active'] + spotrac_nfl_records_pos_pivot['pos_cap_hit_inactive']

# ---------------------------
# D) Merge Position Data with Team Totals
# ---------------------------
# Merge team–season totals into the position-level DataFrame (matched on team and season)
spotrac_nfl_records_merged = spotrac_nfl_records_pos_pivot.merge(df_team, on=['team', 'season'], how='left')

# ---------------------------
# E) Calculate Percentage Columns
# ---------------------------
# 1. Overall percent that the position’s total cap_hit is of the team’s total cap_hit
spotrac_nfl_records_merged['pos_cap_hit_pct'] = (spotrac_nfl_records_merged['pos_cap_hit'] / spotrac_nfl_records_merged['cap_hit_total']) * 100

# 2. For active roster:
#    a) Percent of team’s active cap hit that the position contributes
spotrac_nfl_records_merged['pos_cap_hit_pct_of_active_cap'] = np.where(
    spotrac_nfl_records_merged['cap_hit_active'] > 0,
    (spotrac_nfl_records_merged['pos_cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_active']) * 100,
    0
)
#    b) Percent of the team's total cap hit that comes from the position’s active cap hit
spotrac_nfl_records_merged['pos_cap_hit_active_pct_of_total_cap'] = (spotrac_nfl_records_merged['pos_cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_total']) * 100

# 3. For inactive roster:
#    a) Percent of team’s inactive cap hit that the position contributes
spotrac_nfl_records_merged['pos_cap_hit_pct_of_inactive_cap'] = np.where(
    spotrac_nfl_records_merged['cap_hit_inactive'] > 0,
    (spotrac_nfl_records_merged['pos_cap_hit_inactive'] / spotrac_nfl_records_merged['cap_hit_inactive']) * 100,
    0
)
#    b) Percent of the team's total cap hit that comes from the position’s inactive cap hit
spotrac_nfl_records_merged['pos_cap_hit_inactive_pct_of_total_cap'] = (spotrac_nfl_records_merged['pos_cap_hit_inactive'] / spotrac_nfl_records_merged['cap_hit_total']) * 100

# This column is the quotient of cap_hit_active / cap_hit_total (multiplied by 100 for percentage)
spotrac_nfl_records_merged['cap_hit_active_inactive_total_pct'] = (
    spotrac_nfl_records_merged['cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_total']
) * 100


# ---------------------------
# F) Reorder Columns as Desired
# ---------------------------
# Adjust the list of columns to match your sample output layout
spotrac_nfl_records_final_columns = [
    'team',
    'season',
    'cap_hit_total',
    'cap_hit_active',
    'cap_hit_inactive',
    'cap_hit_active_inactive_total_pct',  # New column added here.
    'pos',                     # Position column (adjust if your field is named differently)
    'position_level_one',      # e.g., offense, defense, special_team
    'position_level_two',      # e.g., offensive_line, secondary, etc.
    'pos_cap_hit',             # Total cap hit for the position
    'pos_cap_hit_pct',         # Percent of team total cap hit that the position represents
    'pos_cap_hit_active',      # Active cap hit for the position
    'pos_cap_hit_pct_of_active_cap',  # Percent of team’s active cap hit that the position contributes
    'pos_cap_hit_active_pct_of_total_cap',  # Percent of team’s overall cap hit that is active for the position
    'pos_cap_hit_inactive',    # Inactive cap hit for the position
    'pos_cap_hit_pct_of_inactive_cap',  # Percent of team’s inactive cap hit that the position contributes
    'pos_cap_hit_inactive_pct_of_total_cap'  # Percent of team’s overall cap hit that is inactive for the position
]

# Reorder and select columns
spotrac_nfl_team_active_inactive_grouped_df = spotrac_nfl_records_merged[spotrac_nfl_records_final_columns]

spotrac_nfl_team_active_inactive_grouped_df = pd.merge(spotrac_nfl_team_active_inactive_grouped_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_active_inactive_grouped_df = spotrac_nfl_team_active_inactive_grouped_df.drop(['nfl_team_name'], axis=1)

spotrac_nfl_team_active_inactive_grouped_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_inactive_grouped_df.csv')
spotrac_nfl_team_active_inactive_grouped_df

/var/folders/kk/54xn9lv90wq5vmw11pj1svvr0000gn/T/ipykernel_89145/1708077745.py:55: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



,team,season,cap_hit_total,cap_hit_active,cap_hit_inactive,cap_hit_active_inactive_total_pct,pos,position_level_one,position_level_two,pos_cap_hit,...,pos_cap_hit_pct_of_inactive_cap,pos_cap_hit_inactive_pct_of_total_cap,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,C,offense,offensive_line,2350000.0,...,0.000000,0.000000,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,CB,defense,secondary,10006561.0,...,17.748349,1.114830,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,DE,defense,defensive_line,8373000.0,...,6.423438,0.403476,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,DT,defense,defensive_line,3335366.0,...,1.201371,0.075462,8,8,0.500,312,348,-36,0.667,0.583
4,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,FB,offense,running_back,426725.0,...,0.000000,0.000000,8,8,0.500,312,348,-36,0.667,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9479,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,RT,offense,offensive_line,10882499.0,...,0.070459,0.015930,12,5,0.706,485,391,94,0.667,0.750
9480,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,S,defense,secondary,11137989.0,...,2.023548,0.457503,12,5,0.706,485,391,94,0.667,0.750
9481,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,T,offense,offensive_line,248334.0,...,0.456459,0.103201,12,5,0.706,485,391,94,0.667,0.750
9482,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,TE,offense,tight_end,9932262.0,...,5.348314,1.209197,12,5,0.706,485,391,94,0.667,0.750


In [255]:
# ---------------------------
# B) Compute Team–Season Totals
# ---------------------------
# 1. Total cap_hit for each team–season (active + inactive)
spotrac_nfl_team_totals = (
    spotrac_nfl_records_df.groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_total'})
)

# 2. Active cap_hit total (table_number == 0)
spotrac_nfl_team_active = (
    spotrac_nfl_records_df[spotrac_nfl_records_df['table_number'] == 0]
    .groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_active'})
)

# 3. Inactive cap_hit total (table_number > 0)
spotrac_nfl_team_inactive = (
    spotrac_nfl_records_df[spotrac_nfl_records_df['table_number'] > 0]
    .groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_inactive'})
)

# Merge team totals into a single DataFrame.
df_team = (
    spotrac_nfl_team_totals
    .merge(spotrac_nfl_team_active, on=['team', 'season'], how='left')
    .merge(spotrac_nfl_team_inactive, on=['team', 'season'], how='left')
).copy()

# Fill missing numeric values with 0.
num_cols = df_team.select_dtypes(include=['number']).columns
df_team[num_cols] = df_team[num_cols].fillna(0)

# ---------------------------
# C) Compute Position-Level Sums by Roster Status (by position_level_one)
# ---------------------------
# Group by team, season, position_level_one, and roster_status; sum cap_hit at the position_level_one level.
spotrac_nfl_records_position_level_one = (
    spotrac_nfl_records_df.groupby(
        ['team', 'season', 'position_level_one', 'roster_status'],
        as_index=False, observed=True
    )['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'pos_level_one_cap_hit_status'})
)

# Pivot so that we have separate columns for active and inactive sums per position_level_one.
spotrac_nfl_records_position_level_one_pivot = spotrac_nfl_records_position_level_one.pivot_table(
    index=['team', 'season', 'position_level_one'],
    columns='roster_status',
    values='pos_level_one_cap_hit_status',
    fill_value=0
).reset_index()
spotrac_nfl_records_position_level_one_pivot.columns.name = None

# Ensure both 'active' and 'inactive' columns exist.
if 'active' not in spotrac_nfl_records_position_level_one_pivot.columns:
    spotrac_nfl_records_position_level_one_pivot['active'] = 0
if 'inactive' not in spotrac_nfl_records_position_level_one_pivot.columns:
    spotrac_nfl_records_position_level_one_pivot['inactive'] = 0

# Rename the pivot columns for clarity.
spotrac_nfl_records_position_level_one_pivot = spotrac_nfl_records_position_level_one_pivot.rename(
    columns={'active': 'position_level_one_cap_hit_active', 'inactive': 'position_level_one_cap_hit_inactive'}
)

# Create a new column for the total position_level_one cap hit (active + inactive).
spotrac_nfl_records_position_level_one_pivot['position_level_one_cap_hit'] = (
    spotrac_nfl_records_position_level_one_pivot['position_level_one_cap_hit_active'] + spotrac_nfl_records_position_level_one_pivot['position_level_one_cap_hit_inactive']
)

# ---------------------------
# D) Merge Position Data with Team Totals
# ---------------------------
spotrac_nfl_records_merged = spotrac_nfl_records_position_level_one_pivot.merge(df_team, on=['team', 'season'], how='left')

# ---------------------------
# E) Calculate Percentage Columns
# ---------------------------
# 1. Overall percent that the position_level_one cap hit is of the team’s total cap hit.
spotrac_nfl_records_merged['position_level_one_cap_hit_pct'] = (
    (spotrac_nfl_records_merged['position_level_one_cap_hit'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# 2. For active roster:
#    a) Percent of team’s active cap hit that the position_level_one contributes.
spotrac_nfl_records_merged['position_level_one_cap_hit_pct_of_active_cap'] = np.where(
    spotrac_nfl_records_merged['cap_hit_active'] > 0,
    (spotrac_nfl_records_merged['position_level_one_cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_active']) * 100,
    0
)
#    b) Percent of the team's total cap hit that comes from the position_level_one’s active cap hit.
spotrac_nfl_records_merged['position_level_one_cap_hit_active_pct_of_total_cap'] = (
    (spotrac_nfl_records_merged['position_level_one_cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# 3. For inactive roster:
#    a) Percent of team’s inactive cap hit that the position_level_one contributes.
spotrac_nfl_records_merged['position_level_one_cap_hit_pct_of_inactive_cap'] = np.where(
    spotrac_nfl_records_merged['cap_hit_inactive'] > 0,
    (spotrac_nfl_records_merged['position_level_one_cap_hit_inactive'] / spotrac_nfl_records_merged['cap_hit_inactive']) * 100,
    0
)
#    b) Percent of the team's total cap hit that comes from the position_level_one’s inactive cap hit.
spotrac_nfl_records_merged['position_level_one_cap_hit_inactive_pct_of_total_cap'] = (
    (spotrac_nfl_records_merged['position_level_one_cap_hit_inactive'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# NEW: Calculate the new column cap_hit_active_inactive_total_pct as (cap_hit_active/cap_hit_total)*100.
spotrac_nfl_records_merged['cap_hit_active_inactive_total_pct'] = (
    (spotrac_nfl_records_merged['cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# ---------------------------
# F) Reorder Columns as Desired
# ---------------------------
# Final column order as in your sample layout.
spotrac_nfl_records_position_level_one_final_columns = [
    'team',
    'season',
    'cap_hit_total',
    'cap_hit_active',
    'cap_hit_inactive',
    'cap_hit_active_inactive_total_pct',  # New column
    'position_level_one',                   # Grouping level used
    'position_level_one_cap_hit',                # Total cap hit for the position_level_one
    'position_level_one_cap_hit_pct',            # Percent of team total cap hit represented by this position_level_one
    'position_level_one_cap_hit_active',         # Active cap hit for this position_level_one
    'position_level_one_cap_hit_pct_of_active_cap',# Percent of team's active cap hit that this position_level_one contributes
    'position_level_one_cap_hit_active_pct_of_total_cap', # Percent of team's total cap hit that is active for this position_level_one
    'position_level_one_cap_hit_inactive',       # Inactive cap hit for this position_level_one
    'position_level_one_cap_hit_pct_of_inactive_cap',# Percent of team's inactive cap hit that this position_level_one contributes
    'position_level_one_cap_hit_inactive_pct_of_total_cap' # Percent of team's total cap hit that is inactive for this position_level_one
]

# Reorder and select columns for the final DataFrame.
spotrac_nfl_team_position_level_one_active_inactive_grouped_df = spotrac_nfl_records_merged[spotrac_nfl_records_position_level_one_final_columns]

spotrac_nfl_team_position_level_one_active_inactive_grouped_df = pd.merge(spotrac_nfl_team_position_level_one_active_inactive_grouped_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_position_level_one_active_inactive_grouped_df = spotrac_nfl_team_position_level_one_active_inactive_grouped_df.drop(['nfl_team_name'], axis=1)

# ---------------------------
# G) Export and Preview
# ---------------------------
# Adjust the export path as needed.
spotrac_nfl_team_position_level_one_active_inactive_grouped_df.to_csv(f'{project_data_exports_path}/spotrac_nfl_team_position_level_one_active_inactive_grouped_df.csv', index=False)

# Preview the first few rows.
spotrac_nfl_team_position_level_one_active_inactive_grouped_df

/var/folders/kk/54xn9lv90wq5vmw11pj1svvr0000gn/T/ipykernel_89145/2921770367.py:52: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



,team,season,cap_hit_total,cap_hit_active,cap_hit_inactive,cap_hit_active_inactive_total_pct,position_level_one,position_level_one_cap_hit,position_level_one_cap_hit_pct,position_level_one_cap_hit_active,...,position_level_one_cap_hit_pct_of_inactive_cap,position_level_one_cap_hit_inactive_pct_of_total_cap,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,defense,49540386.0,41.816691,46212697.0,...,44.717999,2.808879,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,offense,65010861.0,54.875210,61041169.0,...,53.345335,3.350789,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,special_team,3919117.0,3.308099,3775000.0,...,1.936667,0.121648,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2012,117797796.0,88769403.0,29028393.0,75.357440,defense,51539541.0,43.752551,47320889.0,...,14.532847,3.581266,5,11,0.313,250,357,-107,0.167,0.250
4,arizona-cardinals,2012,117797796.0,88769403.0,29028393.0,75.357440,offense,62916588.0,53.410667,38206847.0,...,85.122662,20.976403,5,11,0.313,250,357,-107,0.167,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,washington-commanders,2023,219158252.0,133719895.0,85438357.0,61.015222,offense,96606737.0,44.080812,74230515.0,...,26.189902,10.210075,4,13,0.235,329,518,-189,0.000,0.167
1340,washington-commanders,2023,219158252.0,133719895.0,85438357.0,61.015222,special_team,6803008.0,3.104153,5933334.0,...,1.017896,0.396825,4,13,0.235,329,518,-189,0.000,0.167
1341,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,defense,109383158.0,45.456527,93441088.0,...,29.302878,6.625071,12,5,0.706,485,391,94,0.667,0.750
1342,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,offense,122993603.0,51.112641,87344973.0,...,65.525209,14.814556,12,5,0.706,485,391,94,0.667,0.750


In [256]:
# ---------------------------
# B) Compute Team–Season Totals
# ---------------------------
# 1. Total cap_hit for each team–season (active + inactive)
spotrac_nfl_team_totals = (
    spotrac_nfl_records_df.groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_total'})
)

# 2. Active cap_hit total (table_number == 0)
spotrac_nfl_team_active = (
    spotrac_nfl_records_df[spotrac_nfl_records_df['table_number'] == 0]
    .groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_active'})
)

# 3. Inactive cap_hit total (table_number > 0)
spotrac_nfl_team_inactive = (
    spotrac_nfl_records_df[spotrac_nfl_records_df['table_number'] > 0]
    .groupby(['team', 'season'], as_index=False, observed=True)['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'cap_hit_inactive'})
)

# Merge team totals into a single DataFrame.
df_team = (
    spotrac_nfl_team_totals
    .merge(spotrac_nfl_team_active, on=['team', 'season'], how='left')
    .merge(spotrac_nfl_team_inactive, on=['team', 'season'], how='left')
).copy()

# Fill missing numeric values with 0.
num_cols = df_team.select_dtypes(include=['number']).columns
df_team[num_cols] = df_team[num_cols].fillna(0)

# ---------------------------
# C) Compute Position-Level Sums by Roster Status (by position_level_two)
# ---------------------------
# Group by team, season, position_level_two, and roster_status; sum cap_hit at the position_level_two level.
spotrac_nfl_records_position_level_two = (
    spotrac_nfl_records_df.groupby(
        ['team', 'season', 'position_level_two', 'roster_status'],
        as_index=False, observed=True
    )['cap_hit']
    .sum()
    .rename(columns={'cap_hit': 'pos_level_two_cap_hit_status'})
)

# Pivot so that we have separate columns for active and inactive sums per position_level_two.
spotrac_nfl_records_position_level_two_pivot = spotrac_nfl_records_position_level_two.pivot_table(
    index=['team', 'season', 'position_level_two'],
    columns='roster_status',
    values='pos_level_two_cap_hit_status',
    fill_value=0
).reset_index()
spotrac_nfl_records_position_level_two_pivot.columns.name = None

# Ensure both 'active' and 'inactive' columns exist.
if 'active' not in spotrac_nfl_records_position_level_two_pivot.columns:
    spotrac_nfl_records_position_level_two_pivot['active'] = 0
if 'inactive' not in spotrac_nfl_records_position_level_two_pivot.columns:
    spotrac_nfl_records_position_level_two_pivot['inactive'] = 0

# Rename the pivot columns for clarity.
spotrac_nfl_records_position_level_two_pivot = spotrac_nfl_records_position_level_two_pivot.rename(
    columns={'active': 'position_level_two_cap_hit_active', 'inactive': 'position_level_two_cap_hit_inactive'}
)

# Create a new column for the total position_level_two cap hit (active + inactive).
spotrac_nfl_records_position_level_two_pivot['position_level_two_cap_hit'] = (
    spotrac_nfl_records_position_level_two_pivot['position_level_two_cap_hit_active'] +
    spotrac_nfl_records_position_level_two_pivot['position_level_two_cap_hit_inactive']
)

# ---------------------------
# D) Merge Position Data with Team Totals
# ---------------------------
spotrac_nfl_records_merged = spotrac_nfl_records_position_level_two_pivot.merge(df_team, on=['team', 'season'], how='left')

# ---------------------------
# E) Calculate Percentage Columns
# ---------------------------
# 1. Overall percent that the position_level_two cap hit is of the team’s total cap hit.
spotrac_nfl_records_merged['position_level_two_cap_hit_pct'] = (
    (spotrac_nfl_records_merged['position_level_two_cap_hit'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# 2. For active roster:
#    a) Percent of team’s active cap hit that the position_level_two contributes.
spotrac_nfl_records_merged['position_level_two_cap_hit_pct_of_active_cap'] = np.where(
    spotrac_nfl_records_merged['cap_hit_active'] > 0,
    (spotrac_nfl_records_merged['position_level_two_cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_active']) * 100,
    0
)
#    b) Percent of the team's total cap hit that comes from the position_level_two’s active cap hit.
spotrac_nfl_records_merged['position_level_two_cap_hit_active_pct_of_total_cap'] = (
    (spotrac_nfl_records_merged['position_level_two_cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# 3. For inactive roster:
#    a) Percent of team’s inactive cap hit that the position_level_two contributes.
spotrac_nfl_records_merged['position_level_two_cap_hit_pct_of_inactive_cap'] = np.where(
    spotrac_nfl_records_merged['cap_hit_inactive'] > 0,
    (spotrac_nfl_records_merged['position_level_two_cap_hit_inactive'] / spotrac_nfl_records_merged['cap_hit_inactive']) * 100,
    0
)
#    b) Percent of the team's total cap hit that comes from the position_level_two’s inactive cap hit.
spotrac_nfl_records_merged['position_level_two_cap_hit_inactive_pct_of_total_cap'] = (
    (spotrac_nfl_records_merged['position_level_two_cap_hit_inactive'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# NEW: Calculate the new column cap_hit_active_inactive_total_pct as (cap_hit_active/cap_hit_total)*100.
spotrac_nfl_records_merged['cap_hit_active_inactive_total_pct'] = (
    (spotrac_nfl_records_merged['cap_hit_active'] / spotrac_nfl_records_merged['cap_hit_total']) * 100
)

# ---------------------------
# F) Reorder Columns as Desired
# ---------------------------
# Final column order as in your sample layout (now using position_level_two).
spotrac_nfl_records_position_level_two_final_columns = [
    'team',
    'season',
    'cap_hit_total',
    'cap_hit_active',
    'cap_hit_inactive',
    'cap_hit_active_inactive_total_pct',  # New column
    'position_level_two',                   # Grouping level used
    'position_level_two_cap_hit',           # Total cap hit for the position_level_two
    'position_level_two_cap_hit_pct',       # Percent of team total cap hit represented by this position_level_two
    'position_level_two_cap_hit_active',    # Active cap hit for this position_level_two
    'position_level_two_cap_hit_pct_of_active_cap',  # Percent of team's active cap hit that this position_level_two contributes
    'position_level_two_cap_hit_active_pct_of_total_cap',  # Percent of team's total cap hit that is active for this position_level_two
    'position_level_two_cap_hit_inactive',    # Inactive cap hit for this position_level_two
    'position_level_two_cap_hit_pct_of_inactive_cap',  # Percent of team's inactive cap hit that this position_level_two contributes
    'position_level_two_cap_hit_inactive_pct_of_total_cap'  # Percent of team's total cap hit that is inactive for this position_level_two
]

# Reorder and select columns for the final DataFrame.
spotrac_nfl_team_position_level_two_active_inactive_grouped_df = spotrac_nfl_records_merged[spotrac_nfl_records_position_level_two_final_columns]

spotrac_nfl_team_position_level_two_active_inactive_grouped_df = pd.merge(spotrac_nfl_team_position_level_two_active_inactive_grouped_df, nfl_season_records_df[['w', 'l', 'pct', 'pf', 'pa', 'net_pts', 'div_win_pct', 'conf_win_pct', 'nfl_team_name', 'season']], how='left', left_on=['team', 'season'], right_on=['nfl_team_name', 'season'])
spotrac_nfl_team_position_level_two_active_inactive_grouped_df = spotrac_nfl_team_position_level_two_active_inactive_grouped_df.drop(['nfl_team_name'], axis=1)

# ---------------------------
# G) Export and Preview
# ---------------------------
# Adjust the export path as needed.
spotrac_nfl_team_position_level_two_active_inactive_grouped_df.to_csv(
    f'{project_data_exports_path}/spotrac_nfl_team_position_level_two_active_inactive_grouped_df.csv', index=False
)

# Preview the first few rows.
spotrac_nfl_team_position_level_two_active_inactive_grouped_df

/var/folders/kk/54xn9lv90wq5vmw11pj1svvr0000gn/T/ipykernel_89145/4019007424.py:52: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



,team,season,cap_hit_total,cap_hit_active,cap_hit_inactive,cap_hit_active_inactive_total_pct,position_level_two,position_level_two_cap_hit,position_level_two_cap_hit_pct,position_level_two_cap_hit_active,...,position_level_two_cap_hit_pct_of_inactive_cap,position_level_two_cap_hit_inactive_pct_of_total_cap,w,l,pct,pf,pa,net_pts,div_win_pct,conf_win_pct
0,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,defensive_line,11708366.0,9.882949,11140966.0,...,7.624809,0.478938,8,8,0.500,312,348,-36,0.667,0.583
1,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,linebacker,14205695.0,11.990927,12766149.0,...,19.344842,1.215111,8,8,0.500,312,348,-36,0.667,0.583
2,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,offensive_line,23253666.0,19.628256,21546666.0,...,22.938930,1.440867,8,8,0.500,312,348,-36,0.667,0.583
3,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,quarterback,11707912.0,9.882566,10407912.0,...,17.469601,1.097321,8,8,0.500,312,348,-36,0.667,0.583
4,arizona-cardinals,2011,118470364.0,111028866.0,7441498.0,93.718684,running_back,5064917.0,4.275261,4104225.0,...,12.909928,0.810913,8,8,0.500,312,348,-36,0.667,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,running_back,7759434.0,3.224600,6766159.0,...,1.825724,0.412777,12,5,0.706,485,391,94,0.667,0.750
4028,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,secondary,29756296.0,12.365870,23041546.0,...,12.342281,2.790459,12,5,0.706,485,391,94,0.667,0.750
4029,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,special_team,8255695.0,3.430832,5441944.0,...,5.171913,1.169315,12,5,0.706,485,391,94,0.667,0.750
4030,washington-commanders,2024,240632456.0,186228005.0,54404451.0,77.391059,tight_end,9932262.0,4.127565,7022541.0,...,5.348314,1.209197,12,5,0.706,485,391,94,0.667,0.750


# Collapse Game Results for College Teams

In [257]:
winners = game_results_df[['winner']].drop_duplicates(subset=['winner'], keep='first')
losers = game_results_df[['loser']].drop_duplicates(subset=['loser'], keep='first')
winners_losers_df = pd.concat([winners, losers], ignore_index=True, axis=0)
winners_losers_set = set(winners_losers_df['winner'])
print(len(winners_losers_set))
winners_losers_set


209


{'Abilene Christian',
 'Air Force',
 'Akron',
 'Alabama',
 'Alabama-Birmingham',
 'Albany',
 'Appalachian State',
 'Arizona',
 'Arizona State',
 'Arkansas',
 'Arkansas State',
 'Army',
 'Auburn',
 'Ball State',
 'Baylor',
 'Bethune-Cookman',
 'Boise State',
 'Boston College',
 'Boston University',
 'Bowling Green',
 'Brigham Young',
 'Buffalo',
 'Cal Poly',
 'Cal State Fullerton',
 'Cal State Northridge',
 'California',
 'California-Davis',
 'Central Arkansas',
 'Central Florida',
 'Central Michigan',
 'Charleston Southern',
 'Charlotte',
 'Chattanooga',
 'Cincinnati',
 'Citadel',
 'Clemson',
 'Coastal Carolina',
 'Colgate',
 'Colorado',
 'Colorado State',
 'Connecticut',
 'Delaware',
 'Duke',
 'Duquesne',
 'East Carolina',
 'East Tennessee State',
 'Eastern Illinois',
 'Eastern Kentucky',
 'Eastern Michigan',
 'Eastern Washington',
 'Florida',
 'Florida A&M',
 'Florida Atlantic',
 'Florida International',
 'Florida State',
 'Fordham',
 'Fresno State',
 'Furman',
 'Gardner-Webb',
 'Geo

In [258]:
# Filter the dataset
filtered_game_results_df = filter_games_by_date(game_results_df)

# Create team-season combinations
team_season_combinations = pd.DataFrame(
    [(team, season) for team in winners_losers_set for season in college_seasons],
    columns=['team', 'season']
)

# Calculate wins
wins = filtered_game_results_df[~filtered_game_results_df['tie']].groupby(['season', 'winner'], observed=True).size().reset_index(name='wins').rename(columns={'winner': 'team'})

# Calculate losses
losses = filtered_game_results_df[~filtered_game_results_df['tie']].groupby(['season', 'loser'], observed=True).size().reset_index(name='losses').rename(columns={'loser': 'team'})

# Calculate ties for both teams involved
ties_winner = filtered_game_results_df[filtered_game_results_df['tie']].groupby(['season', 'winner'], observed=True).size().reset_index(name='ties').rename(columns={'winner': 'team'})
ties_loser = filtered_game_results_df[filtered_game_results_df['tie']].groupby(['season', 'loser'], observed=True).size().reset_index(name='ties').rename(columns={'loser': 'team'})
ties = pd.concat([ties_winner, ties_loser]).groupby(['season', 'team'], observed=True).sum().reset_index()

# Merge wins, losses, and ties into the team-season DataFrame
team_season_records_df = team_season_combinations.merge(wins, on=['team', 'season'], how='left') \
                                   .merge(losses, on=['team', 'season'], how='left') \
                                   .merge(ties, on=['team', 'season'], how='left')

# Fill NaN with 0 for missing values
team_season_records_df['wins'] = team_season_records_df['wins'].fillna(0).astype(int)
team_season_records_df['losses'] = team_season_records_df['losses'].fillna(0).astype(int)
team_season_records_df['ties'] = team_season_records_df['ties'].fillna(0).astype(int)

# Calculate winning percentage
team_season_records_df['games_played'] = team_season_records_df['wins'] + team_season_records_df['losses'] + team_season_records_df['ties']
team_season_records_df['winning_pct'] = team_season_records_df.apply(
    lambda row: (row['wins'] + 0.5 * row['ties']) / row['games_played'] if row['games_played'] > 0 else 0,
    axis=1
)

# Keep only teams with more than 4 games played
team_season_records_df = team_season_records_df.loc[team_season_records_df['games_played'] > 4, :].reset_index(drop=True)
team_season_records_df = team_season_records_df.sort_values(['team', 'season'], ascending=[True, True]).reset_index(drop=True)

# Add an index column
team_season_records_df['index'] = team_season_records_df.index

team_season_records_df.to_csv(f"{project_data_exports_path}/team_season_records_df.csv")
team_season_records_df

,team,season,wins,losses,ties,games_played,winning_pct,index
0,Air Force,1990,6,5,0,11,0.545455,0
1,Air Force,1991,9,3,0,12,0.750000,1
2,Air Force,1992,7,4,0,11,0.636364,2
3,Air Force,1993,4,8,0,12,0.333333,3
4,Air Force,1994,8,4,0,12,0.666667,4
...,...,...,...,...,...,...,...,...
4049,Wyoming,2019,8,5,0,13,0.615385,4049
4050,Wyoming,2020,2,4,0,6,0.333333,4050
4051,Wyoming,2021,7,6,0,13,0.538462,4051
4052,Wyoming,2022,7,6,0,13,0.538462,4052


In [259]:
# team_season_combinations = pd.DataFrame([(team, season) for team in winners_losers_set for season in college_seasons], columns=['team', 'season'])
#
# # Calculate wins
# wins = game_results_df[~game_results_df['tie']].groupby(['season', 'winner'], observed=True).size().reset_index(name='wins').rename(columns={'winner': 'team'})
#
# # Calculate losses
# losses = game_results_df[~game_results_df['tie']].groupby(['season', 'loser'], observed=True).size().reset_index(name='losses').rename(columns={'loser': 'team'})
#
# # Calculate ties for both teams involved
# ties_winner = game_results_df[game_results_df['tie']].groupby(['season', 'winner'], observed=True).size().reset_index(name='ties').rename(columns={'winner': 'team'})
# ties_loser = game_results_df[game_results_df['tie']].groupby(['season', 'loser'], observed=True).size().reset_index(name='ties').rename(columns={'loser': 'team'})
# ties = pd.concat([ties_winner, ties_loser]).groupby(['season', 'team'], observed=True).sum().reset_index()
#
# # Merge wins, losses, and ties into the team-season DataFrame
# team_season_records_df = team_season_combinations.merge(wins, on=['team', 'season'], how='left') \
#                                    .merge(losses, on=['team', 'season'], how='left') \
#                                    .merge(ties, on=['team', 'season'], how='left')
#
# # Fill NaN with 0 for missing values
# team_season_records_df['wins'] = team_season_records_df['wins'].fillna(0).astype(int)
# team_season_records_df['losses'] = team_season_records_df['losses'].fillna(0).astype(int)
# team_season_records_df['ties'] = team_season_records_df['ties'].fillna(0).astype(int)
#
# # Calculate winning percentage
# team_season_records_df['games_played'] = team_season_records_df['wins'] + team_season_records_df['losses'] + team_season_records_df['ties']
# team_season_records_df['winning_pct'] = team_season_records_df.apply(
#     lambda row: (row['wins'] + 0.5 * row['ties']) / row['games_played'] if row['games_played'] > 0 else 0,
#     axis=1
# )
#
# team_season_records_df = team_season_records_df.loc[team_season_records_df['games_played'] > 4, :].reset_index(drop=True)
# team_season_records_df = team_season_records_df.sort_values(['team', 'season'], ascending=[True, True]).reset_index(drop=True)
# team_season_records_df['index'] = team_season_records_df.index
#
# team_season_records_df


# Aggregate Offensive Stats For Teams and Seasons

## Passing Offense

In [260]:
filtered_passing_offense_df = passing_offense_df.merge(team_season_records_df[['team', 'season']], on=['team', 'season'], how='inner')

passing_offense_aggregated_stats = filtered_passing_offense_df.groupby(['team', 'season']).agg({
    'cmp': 'sum',
    'att': 'sum',
    'yds': 'sum',
    'td': 'sum',
    'int': 'sum'
}).reset_index()

team_season_records_stats_df = team_season_records_df.merge(
    passing_offense_aggregated_stats,
    on=['team', 'season'],
    how='left'
)

team_season_records_stats_df = team_season_records_stats_df.drop(columns={'index'})

team_season_records_stats_df = team_season_records_stats_df.assign(cmp_pct=team_season_records_stats_df['cmp'] / team_season_records_stats_df['att'])
team_season_records_stats_df = team_season_records_stats_df.assign(td_pct=team_season_records_stats_df['td'] / team_season_records_stats_df['att'])
team_season_records_stats_df = team_season_records_stats_df.assign(int_pct=team_season_records_stats_df['int'] / team_season_records_stats_df['att'])
team_season_records_stats_df = team_season_records_stats_df.assign(yds_per_att=team_season_records_stats_df['yds'] / team_season_records_stats_df['att'])
team_season_records_stats_df = team_season_records_stats_df.assign(adj_yds_per_att=(team_season_records_stats_df['yds'] + (20 * team_season_records_stats_df['td']) - (45 * team_season_records_stats_df['int']))/ team_season_records_stats_df['att'])
team_season_records_stats_df = team_season_records_stats_df.assign(rating=(8.4 * team_season_records_stats_df['yds'] + 330 * team_season_records_stats_df['td'] - 200 * team_season_records_stats_df['int'] + 100 * team_season_records_stats_df['cmp'])/ team_season_records_stats_df['att'])
team_season_records_stats_df = team_season_records_stats_df.assign(yds_per_cmp=team_season_records_stats_df['yds'] / team_season_records_stats_df['cmp'])
team_season_records_stats_df = team_season_records_stats_df.assign(yds_per_g=team_season_records_stats_df['yds'] / team_season_records_stats_df['games_played'])

team_season_records_stats_df.to_csv(f'{project_data_exports_path}/team_season_records_stats_df.csv')
team_season_records_stats_df

,team,season,wins,losses,ties,games_played,winning_pct,cmp,att,yds,td,int,cmp_pct,td_pct,int_pct,yds_per_att,adj_yds_per_att,rating,yds_per_cmp,yds_per_g
0,Air Force,1990,6,5,0,11,0.545455,30.0,90.0,386.0,0.0,6.0,0.333333,0.000000,0.066667,4.288889,1.288889,56.026667,12.866667,35.090909
1,Air Force,1991,9,3,0,12,0.750000,32.0,103.0,795.0,7.0,3.0,0.310680,0.067961,0.029126,7.718447,7.766990,112.504854,24.843750,66.250000
2,Air Force,1992,7,4,0,11,0.636364,47.0,122.0,705.0,1.0,8.0,0.385246,0.008197,0.065574,5.778689,2.991803,76.655738,15.000000,64.090909
3,Air Force,1993,4,8,0,12,0.333333,53.0,151.0,803.0,6.0,13.0,0.350993,0.039735,0.086093,5.317881,2.238411,75.663576,15.150943,66.916667
4,Air Force,1994,8,4,0,12,0.666667,80.0,163.0,1563.0,10.0,4.0,0.490798,0.061350,0.024540,9.588957,9.711656,144.964417,19.537500,130.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,Wyoming,2019,8,5,0,13,0.615385,120.0,256.0,1770.0,11.0,9.0,0.468750,0.042969,0.035156,6.914062,6.191406,112.101562,14.750000,136.153846
4050,Wyoming,2020,2,4,0,6,0.333333,62.0,133.0,920.0,1.0,5.0,0.466165,0.007519,0.037594,6.917293,5.375940,99.684211,14.838710,153.333333
4051,Wyoming,2021,7,6,0,13,0.538462,162.0,297.0,2115.0,15.0,12.0,0.545455,0.050505,0.040404,7.121212,6.313131,122.949495,13.055556,162.692308
4052,Wyoming,2022,7,6,0,13,0.538462,156.0,304.0,1719.0,11.0,12.0,0.513158,0.036184,0.039474,5.654605,4.601974,102.860526,11.019231,132.230769


## Receiving Offense

In [261]:
filtered_receiving_offense_df = receiving_offense_df.merge(team_season_records_df[['team', 'season']], on=['team', 'season'], how='inner')

receiving_offense_aggregated_stats = filtered_receiving_offense_df.groupby(['team', 'season']).agg({
    'receptions': 'sum',
    'receiving_yds': 'sum',
    'receiving_td': 'sum',
    'rushing_att': 'sum',
    'rushing_yds': 'sum',
    'rushing_td': 'sum',
    'scrimmage_plays': 'sum',
    'scrimmage_yds': 'sum',
    'scrimmage_td': 'sum',
}).reset_index()

receiving_offense_aggregated_stats = receiving_offense_aggregated_stats.rename(
    columns={
        'receptions': 'receptions_receivers',
        'receiving_yds': 'receiving_yds_receivers',
        'receiving_td': 'receiving_td_receivers',
        'rushing_att': 'rushing_att_receivers',
        'rushing_yds': 'rushing_yds_receivers',
        'rushing_td': 'rushing_td_receivers',
        'scrimmage_plays': 'scrimmage_plays_receivers',
        'scrimmage_yds': 'scrimmage_yds_receivers',
        'scrimmage_td': 'scrimmage_td_receivers',
    }
)

team_season_records_stats_df = team_season_records_stats_df.merge(
    receiving_offense_aggregated_stats,
    on=['team', 'season'],
    how='left',
)

team_season_records_stats_df = team_season_records_stats_df.assign(receiving_yds_per_reception_receivers=team_season_records_stats_df['receiving_yds_receivers'] / team_season_records_stats_df['receptions_receivers'])
team_season_records_stats_df = team_season_records_stats_df.assign(reciving_yds_per_g_receivers=team_season_records_stats_df['receiving_yds_receivers'] / team_season_records_stats_df['games_played'])
team_season_records_stats_df = team_season_records_stats_df.assign(rushing_yds_per_rushing_att_receivers=team_season_records_stats_df['rushing_yds_receivers'] / team_season_records_stats_df['rushing_att_receivers'])
team_season_records_stats_df = team_season_records_stats_df.assign(rushing_yds_per_g_receivers=team_season_records_stats_df['rushing_yds_receivers'] / team_season_records_stats_df['games_played'])
team_season_records_stats_df = team_season_records_stats_df.assign(avg_scrimmage_yds_per_play_receivers=team_season_records_stats_df['scrimmage_yds_receivers'] / team_season_records_stats_df['scrimmage_plays_receivers'])

team_season_records_stats_df.to_csv(f'{project_data_exports_path}/team_season_records_stats_df.csv')
team_season_records_stats_df

,team,season,wins,losses,ties,games_played,winning_pct,cmp,att,yds,...,rushing_yds_receivers,rushing_td_receivers,scrimmage_plays_receivers,scrimmage_yds_receivers,scrimmage_td_receivers,receiving_yds_per_reception_receivers,reciving_yds_per_g_receivers,rushing_yds_per_rushing_att_receivers,rushing_yds_per_g_receivers,avg_scrimmage_yds_per_play_receivers
0,Air Force,1990,6,5,0,11,0.545455,30.0,90.0,386.0,...,1378.0,15.0,284.0,1764.0,15.0,12.866667,35.090909,5.425197,125.272727,6.211268
1,Air Force,1991,9,3,0,12,0.750000,32.0,103.0,795.0,...,2142.0,20.0,443.0,2937.0,27.0,24.843750,66.250000,5.211679,178.500000,6.629797
2,Air Force,1992,7,4,0,11,0.636364,47.0,122.0,705.0,...,1893.0,14.0,420.0,2640.0,16.0,15.244898,67.909091,5.102426,172.090909,6.285714
3,Air Force,1993,4,8,0,12,0.333333,53.0,151.0,803.0,...,2121.0,18.0,487.0,2924.0,24.0,15.150943,66.916667,4.887097,176.750000,6.004107
4,Air Force,1994,8,4,0,12,0.666667,80.0,163.0,1563.0,...,2293.0,20.0,496.0,3856.0,30.0,19.537500,130.250000,5.512019,191.083333,7.774194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,Wyoming,2019,8,5,0,13,0.615385,120.0,256.0,1770.0,...,1820.0,12.0,502.0,3590.0,23.0,14.750000,136.153846,4.764398,140.000000,7.151394
4050,Wyoming,2020,2,4,0,6,0.333333,62.0,133.0,920.0,...,1107.0,9.0,255.0,2048.0,10.0,14.936508,156.833333,5.765625,184.500000,8.031373
4051,Wyoming,2021,7,6,0,13,0.538462,162.0,297.0,2115.0,...,1974.0,15.0,533.0,4089.0,30.0,13.055556,162.692308,5.320755,151.846154,7.671670
4052,Wyoming,2022,7,6,0,13,0.538462,156.0,304.0,1719.0,...,1545.0,10.0,447.0,3264.0,21.0,11.019231,132.230769,5.309278,118.846154,7.302013


## Rushing Offense

In [262]:
filtered_rushing_offense_df = rushing_offense_df.merge(team_season_records_df[['team', 'season']], on=['team', 'season'], how='inner')

rushing_offense_aggregated_stats = filtered_rushing_offense_df.groupby(['team', 'season']).agg({
    'rushing_att': 'sum',
    'rushing_yds': 'sum',
    'rushing_td': 'sum',
    'receptions': 'sum',
    'receiving_yds': 'sum',
    'receiving_td': 'sum',
    'scrimmage_plays': 'sum',
    'scrimmage_yds': 'sum',
    'scrimmage_td': 'sum',
}).reset_index()

rushing_offense_aggregated_stats = rushing_offense_aggregated_stats.rename(
    columns={
        'rushing_att': 'rushing_att_backs',
        'rushing_yds': 'rushing_yds_backs',
        'rushing_td': 'rushing_td_backs',
        'receptions': 'receptions_backs',
        'receiving_yds': 'receiving_yds_backs',
        'receiving_td': 'receiving_td_backs',
        'scrimmage_plays': 'scrimmage_plays_backs',
        'scrimmage_yds': 'scrimmage_yds_backs',
        'scrimmage_td': 'scrimmage_td_backs',
    }
)

team_season_records_stats_df = team_season_records_stats_df.merge(
    rushing_offense_aggregated_stats,
    on=['team', 'season'],
    how='left',
)

team_season_records_stats_df = team_season_records_stats_df.assign(rushing_yds_per_rushing_att_backs=team_season_records_stats_df['rushing_yds_backs'] / team_season_records_stats_df['rushing_att_backs'])
team_season_records_stats_df = team_season_records_stats_df.assign(rushing_yds_per_g_backs=team_season_records_stats_df['rushing_yds_backs'] / team_season_records_stats_df['games_played'])
team_season_records_stats_df = team_season_records_stats_df.assign(receiving_yds_per_reception_att_backs=team_season_records_stats_df['receiving_yds_backs'] / team_season_records_stats_df['receptions_backs'])
team_season_records_stats_df = team_season_records_stats_df.assign(receiving_yds_per_g_backs=team_season_records_stats_df['receiving_yds_backs'] / team_season_records_stats_df['games_played'])
team_season_records_stats_df = team_season_records_stats_df.assign(avg_scrimmage_yds_per_play_backs=team_season_records_stats_df['scrimmage_yds_backs'] / team_season_records_stats_df['scrimmage_plays_backs'])

team_season_records_stats_df['index'] = team_season_records_stats_df.index

team_season_records_stats_df.to_csv(f'{project_data_exports_path}/team_season_records_stats_df.csv')
team_season_records_stats_df

,team,season,wins,losses,ties,games_played,winning_pct,cmp,att,yds,...,receiving_td_backs,scrimmage_plays_backs,scrimmage_yds_backs,scrimmage_td_backs,rushing_yds_per_rushing_att_backs,rushing_yds_per_g_backs,receiving_yds_per_reception_att_backs,receiving_yds_per_g_backs,avg_scrimmage_yds_per_play_backs,index
0,Air Force,1990,6,5,0,11,0.545455,30.0,90.0,386.0,...,0.0,665.0,3057.0,28.0,4.484756,267.454545,12.777778,10.454545,4.596992,0
1,Air Force,1991,9,3,0,12,0.750000,32.0,103.0,795.0,...,5.0,787.0,4698.0,39.0,5.338158,338.083333,23.740741,53.416667,5.969504,1
2,Air Force,1992,7,4,0,11,0.636364,47.0,122.0,705.0,...,1.0,628.0,3041.0,27.0,4.453642,244.545455,14.625000,31.909091,4.842357,2
3,Air Force,1993,4,8,0,12,0.333333,53.0,151.0,803.0,...,5.0,752.0,4092.0,34.0,4.842475,286.916667,15.829268,54.083333,5.441489,3
4,Air Force,1994,8,4,0,12,0.666667,80.0,163.0,1563.0,...,9.0,786.0,4967.0,45.0,5.079167,304.750000,19.848485,109.166667,6.319338,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,Wyoming,2019,8,5,0,13,0.615385,120.0,256.0,1770.0,...,5.0,587.0,3480.0,24.0,4.950820,185.846154,10.747475,81.846154,5.928450,4049
4050,Wyoming,2020,2,4,0,6,0.333333,62.0,133.0,920.0,...,0.0,314.0,1986.0,16.0,4.900369,221.333333,15.302326,109.666667,6.324841,4050
4051,Wyoming,2021,7,6,0,13,0.538462,162.0,297.0,2115.0,...,13.0,654.0,4468.0,37.0,5.185393,213.000000,14.158333,130.692308,6.831804,4051
4052,Wyoming,2022,7,6,0,13,0.538462,156.0,304.0,1719.0,...,2.0,529.0,2995.0,18.0,5.051064,182.615385,10.525424,47.769231,5.661626,4052
